<a href="https://colab.research.google.com/github/mech0s/nodehenge/blob/main/experiment/experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
revision = 5
print ("Revision", revision)

Revision 5


##  Install steps
Remeber to re-run for each fresh google colab session

In [1]:
%pip install rdfpandas

  Using cached rdfpandas-1.1.6-py3-none-any.whl (5.8 kB)
  Using cached rdflib-6.3.2-py3-none-any.whl (528 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
                                              0.0/102.6 kB ? eta -:--:--
     -------------------------------------- 102.6/102.6 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


##   Imports

In [1]:
import numpy as np
import pandas as pd


### Optional    google  colab   enhancements

In [2]:
#optional
from google.colab import data_table
data_table.enable_dataframe_formatter()

##  Code

In [ ]:
sourceURI = "https://dodcio.defense.gov/Portals/0/Documents/Library/DevSecOpsActivitesToolsGuidebookTables.xlsx"
xls = pd.ExcelFile(sourceURI)

In [ ]:
xls.sheet_names

['Cover',
 'Overview',
 'Continuous Activities',
 'Plan',
 'Develop',
 'Build',
 'Test',
 'Release',
 'Deliver',
 'Deploy',
 'Operate',
 'Monitor',
 'Feedback',
 'Tools']

In [ ]:
phaseNames = [ 'Plan',
 'Develop',
 'Build',
 'Test',
 'Release',
 'Deliver',
 'Deploy',
 'Operate',
 'Monitor',
 'Feedback']

One named worksheet per phase: Pull these into a list of DataFrames, adjusting column names to create valid identifiers. Turn NaN entries into blank strings.

In [ ]:
phaseDataList = []
for phName in phaseNames:
  df = pd.read_excel(xls,phName)
  df = df.rename(columns={"Activities":"Activity",
                     "Security / Testing / CM": "SecurityTestingCM",
                     "Tool Dependencies": "ToolDependency",
                     "Tool Dependency": "ToolDependency"
                     })
  phaseDataList.append(df.replace(np.nan, ""))

List of dataframes - concatenated into one dataframe, with the addition of a Phase column for identification.

In [ ]:
phaseActivityDataFrame = pd.concat(phaseDataList,keys=phaseNames, names=["Phase","IDinPhase"])

In [ ]:
phaseActivityDataFrame

Activity   Baseline  \
Phase    IDinPhase                                                      
Plan     0                      Change management planning   REQUIRED   
         1                    Configuration identification   REQUIRED   
         2          Configuration management (CM) planning   REQUIRED   
         3                                 Database design  PREFERRED   
         4                                   Design review  PREFERRED   
...                                                    ...        ...   
Monitor  12                                     Test Audit   REQUIRED   
         13                       Test configuration audit  PREFERRED   
         14                     User Evaluation / Feedback   REQUIRED   
Feedback 0                          Revise Product Backlog   REQUIRED   
         1                      User Evaluation / Feedback   REQUIRED   

                                                                 SSDF  \
Phase    IDinPhase                                                      
Plan     0                             PO.1.1, PS.1.1, PS.3.1, PW.6.1   
         1             PO.2.1, PS.1.1, PW.2.1, PW.4.1, PW.4.2, PW.6.2   
         2                     PO.3.1, PO.3.3, PO.4.1, PO.4.2, PW.2.1   
         3                     PO.1.2, PO.3.1, PO.5.2, PW.1.1, PW.5.1   
         4                     PO.1.2, PW.1.2, PW.2.1, PW.8.2, RV.2.2   
...                                                               ...   
Monitor  12                      PO 2.1, \nPS 2.1, \nPW 1.2, \nPW 2.1   
         13                                                    PO 3.3   
         14                                                    PO.4.2   
Feedback 0          PO.1.1, PO.1.2; PO.1.3, PO.3.1, PO.4.1, RV.2.2...   
         1                                                     PO.4.2   

                                                          Description  \
Phase    IDinPhase                                                      
Plan     0                            Plan the change control process   
         1          Discover or manual input configuration items i...   
         2          Plan the configuration control process;\nIdent...   
         3          Data modeling; \nDatabase selection;\nDatabase...   
         4                     Review and approve plans and documents   
...                                                               ...   
Monitor  12         Test audit keeps who performs what test at wha...   
         13                      Track test and security scan results   
         14         Collect and analyze user evaluations and/or fe...   
Feedback 0          Update the product backlog with new features, ...   
         1          Collect and analyze user evaluations and/or fe...   

                                                               Inputs  \
Phase    IDinPhase                                                      
Plan     0          Organizational policy;\nSoftware development b...   
         1          IT infrastructure asset;\nSoftware system comp...   
         2          Software development, security and operations ...   
         3                         System requirement;\nSystem design   
         4                                Plans and design documents;   
...                                                               ...   
Monitor  12                            Test activity and test results   
         13                                             Test results;   
         14                   Surveys;\nBug reports;\nTrouble tickets   
Feedback 0          Requirements;\nImprovements;\nBugs;\nVulnerabi...   
         1                    Surveys;\nBug reports;\nTrouble tickets   

                                                              Outputs  \
Phase    IDinPhase                                                      
Plan     0          Change control procedures;\nReview procedures;...   
         1                     

In [ ]:
tmpdf =  phaseActivityDataFrame.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
variableColumns = [
                   #'Activity' ,
                   "SSDF",
                    'Baseline',
                    'Description',
                    'Inputs',
                    'Outputs',
                   # 'SecurityTestingCM',
                   # 'ToolDependency'
                   ]

tmpdf.columns.difference(variableColumns)



Index(['Activity', 'Outputs', 'SecurityTestingCM', 'ToolDependency'], dtype='object')

In [ ]:
tmpdf[tmpdf.duplicated(keep=False, subset=tmpdf.columns.difference(variableColumns) )]

Activity   Baseline  \
Phase    IDinPhase                                                           
Plan     14                                          Test Audit   REQUIRED   
Develop  5                                       Component Test  PREFERRED   
         13                Mission Based Cyber Risk Assessments   REQUIRED   
         18                                          Test Audit   REQUIRED   
Build    0                                   API Security Tests   REQUIRED   
         3                                       Component Test   REQUIRED   
         7                 Mission Based Cyber Risk Assessments   REQUIRED   
         10                           Software Integration Test   REQUIRED   
         14                                          Test Audit   REQUIRED   
Test     0                                   API Security Tests   REQUIRED   
         6                 Mission Based Cyber Risk Assessments   REQUIRED   
         9                   SBOM Software Composition Analysis   REQUIRED   
         11         Software Factory Risk Continuous Monitoring   REQUIRED   
         12                           Software Integration Test   REQUIRED   
         16                                          Test Audit   REQUIRED   
Release  2                 Mission Based Cyber Risk Assessments   REQUIRED   
         5                   SBOM Software Composition Analysis   REQUIRED   
         6          Software Factory Risk Continuous Monitoring   REQUIRED   
         7                                           Test Audit   REQUIRED   
Deliver  3                 Mission Based Cyber Risk Assessments   REQUIRED   
         6                   SBOM Software Composition Analysis   REQUIRED   
         7          Software Factory Risk Continuous Monitoring   REQUIRED   
         8                                           Test Audit   REQUIRED   
Deploy   11                Mission Based Cyber Risk Assessments   REQUIRED   
         18                  SBOM Software Composition Analysis   REQUIRED   
         19         Software Factory Risk Continuous Monitoring   REQUIRED   
         20                                          Test Audit   REQUIRED   
         21                          User Evaluation / Feedback   REQUIRED   
Operate  5                 Mission Based Cyber Risk Assessments   REQUIRED   
         9                   SBOM Software Composition Analysis   REQUIRED   
         10         Software Factory Risk Continuous Monitoring   REQUIRED   
         12                                          Test Audit   REQUIRED   
         13                          User Evaluation / Feedback   REQUIRED   
Monitor  7                 Mission Based Cyber Risk Assessments   REQUIRED   
         12                                          Test Audit   REQUIRED   
         14                          User Evaluation / Feedback   REQUIRED   
Feedback 1                           User Evaluation / Feedback   REQUIRED   

                                                                 SSDF  \
Phase    IDinPhase                                                      
Plan     14                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Develop  5                                             RV.8.1, RV.8.2   
         13         PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         18                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Build    0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         3                                           RV.8.1, \nRV.8.2   
         7          PW.7.2, \nRV.1.1,\nRV.1.2, \nRV.2.1, \nRV.3.1,...   
         10                                          RV.8.1, \nRV.8.2   
         14                      PO.2.1, \nPS 2.1, \nPW.1.2, \nPW.2.1   
Test     0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         6          PW.7.2, \nRV.1.1, \nRV.1.2, \nRV.2.1, \nRV.3.1...   
         9                                                     

In [ ]:
variableColumns = [ ]#,'Description']
phaseActivityDataFrame[phaseActivityDataFrame.duplicated(keep=False, subset=phaseActivityDataFrame.columns.difference(variableColumns))]


Activity  Baseline  \
Phase    IDinPhase                                                          
Plan     14                                          Test Audit  REQUIRED   
Develop  13                Mission Based Cyber Risk Assessments  REQUIRED   
         18                                          Test Audit  REQUIRED   
Build    0                                   API Security Tests  REQUIRED   
         14                                          Test Audit  REQUIRED   
Test     0                                   API Security Tests  REQUIRED   
         6                 Mission Based Cyber Risk Assessments  REQUIRED   
         9                   SBOM Software Composition Analysis  REQUIRED   
         11         Software Factory Risk Continuous Monitoring  REQUIRED   
Release  2                 Mission Based Cyber Risk Assessments  REQUIRED   
         5                   SBOM Software Composition Analysis  REQUIRED   
         6          Software Factory Risk Continuous Monitoring  REQUIRED   
Deliver  3                 Mission Based Cyber Risk Assessments  REQUIRED   
         6                   SBOM Software Composition Analysis  REQUIRED   
         7          Software Factory Risk Continuous Monitoring  REQUIRED   
         8                                           Test Audit  REQUIRED   
Deploy   18                  SBOM Software Composition Analysis  REQUIRED   
         19         Software Factory Risk Continuous Monitoring  REQUIRED   
         20                                          Test Audit  REQUIRED   
         21                          User Evaluation / Feedback  REQUIRED   
Operate  5                 Mission Based Cyber Risk Assessments  REQUIRED   
         9                   SBOM Software Composition Analysis  REQUIRED   
         10         Software Factory Risk Continuous Monitoring  REQUIRED   
Monitor  7                 Mission Based Cyber Risk Assessments  REQUIRED   
         14                          User Evaluation / Feedback  REQUIRED   
Feedback 1                           User Evaluation / Feedback  REQUIRED   

                                                                 SSDF  \
Phase    IDinPhase                                                      
Plan     14                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Develop  13         PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         18                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Build    0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         14                      PO.2.1, \nPS 2.1, \nPW.1.2, \nPW.2.1   
Test     0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         6          PW.7.2, \nRV.1.1, \nRV.1.2, \nRV.2.1, \nRV.3.1...   
         9                                                     PS.3.2   
         11                                                    PS.3.2   
Release  2          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         5                                                     PS.3.2   
         6                                                     PS.3.2   
Deliver  3          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         6                                                     PS.3.2   
         7                                                     PS.3.2   
         8                           PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Deploy   18                                                    PS.3.2   
         19                                                    PS.3.2   
         20                      PO.2.1, \nPS 2.1, \nPW.1.2, \nPW.2.1   
         21                                                    PO.4.2   
Operate  5          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         9                                                     PS.3.2   
         10                                                    PS.3.2   
Monitor  7          PW.7.2, \nRV.1.1, \nRV.1.2, \nRV.2.1, \nRV.3.1...   


In [ ]:
"   sdfsdf df   ".strip()

'sdfsdf df'

In [ ]:
import rdfpandas

In [ ]:
graph = rdfpandas.graph.to_graph(phaseActivityDataFrame)

In [ ]:
print(graph.serialize(format='turtle'))


"('Build', 0)" "Activities" "API Security Tests" ;
    "Baseline" "REQUIRED" ;
    "Description" "Closed-box test, evaluating the Application Programming Interfaces' compliance with security requirements" ;
    "Inputs" """Executable system/application
Test Plan
Test Cases
Test data""" ;
    "Outputs" "Test results" ;
    "SSDF" """PO.3.2, 
PO.4.1, 
PW.1.3, 
PW.4.4, 
RV1.2""" ;
    "Security" "Security" ;
    "Tool" "Test tool suite" .

"('Build', 1)" "Activities" "Build" ;
    "Baseline" "REQUIRED" ;
    "Description" "Compile and link" ;
    "Inputs" """Source code;
dependencies""" ;
    "Outputs" """Binary artifacts
Build Report""" ;
    "SSDF" """PO.3.1, 
PO.3.2, 
PO.3.3, 
PO.4.1""" ;
    "Tool" """Build tool;
Lint tool;
Artifact repository""" .

"('Build', 2)" "Activities" "Build configuration control and audit" ;
    "Baseline" "REQUIRED" ;
    "Description" """Track build results, SAST and dependency checking report;
""" ;
    "Inputs" """Build results;
SAST report;
Dependency 

## Experiments

In [6]:
# files written during google colab session are saved to session storage

f = open("textfile.txt", "a")
f.write("file content!")
f.close()